<a href="https://colab.research.google.com/github/EssaAlsharif/Data-Driven-Insights-into-Taekwondo-Player-Behavior/blob/main/Data_Driven_Insights_into_Taekwondo_Player_Behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Data Driven Insights into Taekwondo Player Behavior
In this project, I am exploring Taekwondo players’ behaviour patterns using Machine Learning and feature engineering on Arabic-language data. The goal is to extract actionable insights for the club, while tackling challenges in data preprocessing and analysis.

### **Initialization**

In [7]:
# GitHub
!git clone https://github.com/EssaAlsharif/Data-Driven-Insights-into-Taekwondo-Player-Behavior
%cd Data-Driven-Insights-into-Taekwondo-Player-Behavior
!ls

Cloning into 'Data-Driven-Insights-into-Taekwondo-Player-Behavior'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 55.82 KiB | 937.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/Data-Driven-Insights-into-Taekwondo-Player-Behavior/Data-Driven-Insights-into-Taekwondo-Player-Behavior/Data-Driven-Insights-into-Taekwondo-Player-Behavior
 Data_Driven_Insights_into_Taekwondo_Player_Behavior.ipynb
 README.md
'Registeration Data.csv'


In [8]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
!pip install gdown
import gdown

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

import warnings
warnings.filterwarnings("ignore")

# magic word for producing visualizations in notebook
%matplotlib inline

pd.options.display.max_rows = None
pd.options.display.max_columns = None

### **Loading Data**

In [9]:
# Load data.
data = pd.read_csv("Registeration Data.csv")
print('the shape of the data is:', data.shape)
data.head(5)

the shape of the data is: (194, 12)


,مستمر/ متوقف,اشتراك في,تاريخ ميلاد المشترك,ترتيب المشترك بين الأخوة,الصف,المدرسة,وظيفة ولي الأمر,مكان عمل ولي الأمر,هل مارس المشترك ألعاب قتالية سابقاً؟,حالة المشترك الصحية,هل أجري للمشترك عمليات جراحية؟,المواصلات
0,متوقف,تايكواندو,3-10-2011,الأول,الثامن,عايش السيد,أستاذ,مدرسة صادق نيروخ,لم يمارس,العمر العقلي أقل من أقرانه,لا,نعم
1,متوقف,تايكواندو,30/7/2017,الأول,الثاني,مدرسه الرساله الخاصه,موظف حكومي,المؤتمر الوطني الشعبي للقدس - رام الله,لم يمارس,جيدة,لا,نعم
2,متوقف,تايكواندو,١٤/٦/٢٠١٥,الأول,الثالث,جمعية الشبان المسلمين,محاسب /,محلات أبو عيشة لقطع السيارات الالمانية /شارع ا...,لم يمارس,جيدة,عملية لوز,نعم
3,متوقف,تايكواندو,17/10/2016,الثاني,الثاني,جمعية الشبان المسلمين,محاسب,محلات ابو عيشة لقطع السيارات,لم يمارس,جيدة,لا,نعم
4,متوقف,تايكواندو,٢٥/٦/٢٠١٨,الأول,الأول,جمعية الشبان المسلمين,تاجر ملابس,الخليل,0595422122,0595422123,لا,نعم


In [ ]:
data.info()

### **Renaming form Arabic to English**

In [10]:
# Renaming Columns from English to Arabic
data = data.rename(columns={
    'مستمر/ متوقف': 'player_status',
    'اشتراك في': 'desired_activity',
    'تاريخ ميلاد المشترك': 'birthday',
    'ترتيب المشترك بين الأخوة': 'family_order',
    'الصف': 'school_grade',
    'المدرسة': 'school',
    'وظيفة ولي الأمر': 'guardian_job',
    'مكان عمل ولي الأمر': 'guardian_job_location',
    'هل مارس المشترك ألعاب قتالية سابقاً؟': 'previous_fighting_experience',
    'حالة المشترك الصحية': 'has_health_condition',
    'هل أجري للمشترك عمليات جراحية؟': 'had_surgery',
    'ملاحظات خاصة بالمشترك': 'special_notes',
    'المواصلات':'needs_transportaion',
    'عنوان السكن':'address'
})

In [12]:
data['player_status'].value_counts()

,count
player_status,
متوقف,130
مستمر,45
نصاب,16


In [13]:
# player_status: may be  stopped, continuous, or swindler.
data['player_status'] = data['player_status'].str.lower().map({
    'متوقف': 'stopped',
    'مستمر': 'continuous',
    'نصاب': 'swindler'
})

In [14]:
data['player_status'].value_counts()

,count
player_status,
stopped,130
continuous,45
swindler,16


In [16]:
data['desired_activity'].value_counts()

,count
desired_activity,
تايكواندو,170
مخيم,11
ننشاكو,2
جمباز,2
كراتيه,2
ضرب البوكس او الملاكمة,1
كرتيه,1
ما ترونه الأنسب لعمره وشخصيته.بتوقع تايكواندو,1
كرتيه,1


In [17]:
# desired_activity: may be  taekwondo, camp, gumbaz, karate, or boxing.
data['desired_activity'] = data['desired_activity'].str.lower().map({
    'تايكواندو': 'taekwondo',
    'مخيم': 'camp',
    'جمباز': 'gumbaz',
    'كراتيه': 'karate',
    'ضرب البوكس او الملاكمة': 'boxing',
    'كرتيه': 'karate',
    'ما ترونه الأنسب لعمره وشخصيته.بتوقع تايكواندو': 'taekwondo',
    'كرتيه': 'karate',
    '.':'taekwondo'
})

In [18]:
data['desired_activity'].value_counts()

,count
desired_activity,
taekwondo,171
camp,11
karate,3
gumbaz,2
boxing,1


In [30]:
# remove all rows other than taekwondo.
data = data[data['desired_activity'] == 'taekwondo']

In [31]:
data['desired_activity'].value_counts()

,count
desired_activity,
taekwondo,171


In [32]:
# remove the column since there is no need to it now.
data = data.drop(columns=['desired_activity'])

In [33]:
data['family_order'].value_counts()

,count
family_order,
1.0,73
2.0,34
3.0,25
4.0,13
5.0,13
6.0,6
7.0,2
8.0,1


In [20]:
data['family_order'] = data['family_order'].str.lower().map({
    'الأول': 1,
    'الاول': 1,
    'الثاني': 2,
    'الثالث': 3,
    'الرابع': 4,
    'الخامس': 5,
    'السادس': 6,
    'السادس': 6,
    '6': 6,
    'السابع': 7,
    'الثامن': 8
})

In [21]:
data['family_order'].value_counts()

,count
family_order,
1.0,76
2.0,39
3.0,31
4.0,16
5.0,16
6.0,7
7.0,2
8.0,1


In [22]:
data['school_grade'].value_counts()

,count
school_grade,
الأول,40
الثاني,37
الثالث,21
الرابع,20
السادس,15
الثامن,14
الخامس,13
السابع,11
تمهيدي,4


In [23]:
# school_grade: from 1-10 are school grades, 100 is high education, and -1 is kindergaten
data['school_grade'] = data['school_grade'].str.lower().map({
    'الأول': 1,
    'الثاني': 2,
    'الثالث': 3,
    'الرابع': 4,
    'الخامس': 5,
    'السادس': 6,
    'السابع': 7,
    'الثامن': 8,
    'التاسع': 9,
    'عاشر': 10,
    'جامعة': 100,
    'تمهيدي': -1,
    'روضة': -1,
    'روضه': -1,
    'مدخلش اشي اسعان': -1,
    'تمهيدي روضة': -1
})

In [24]:
data['school_grade'].value_counts()

,count
school_grade,
1.0,40
2.0,37
3.0,21
4.0,20
6.0,15
8.0,14
5.0,13
7.0,11
-1.0,10


In [25]:
data['school'].value_counts()

,count
school,
الرحمه,7
جمعية الشبان المسلمين,4
عبد الحي شاهين,4
انس بن مالك,4
جمعية الشبان المسلمين,3
جبريل زلوم,3
نظيرة,3
رواد المعرفة,3
انصار الابداع,3


In [26]:
def classify_school(name):

    if pd.isna(name):
        return 'other'

    if (
        'الحي' in name or 'السعدية' in name or 'عايش' in name or 'الفاروق' in name or 'علي' in name or
        'الدجاني' in name or 'شاور' in name or 'ابراهيم' in name or 'راضي' in name or 'فهد' in name or
        'الهاجر' in name or 'الغدير' in name or 'غزالة' in name or 'شاهين' in name or 'غازي' in name or
        'الضبعات' in name or 'وداد' in name or 'المحمدية' in name or 'زلوم' in name or 'الدويك' in name or
        'تيسير' in name or 'رشد' in name
    ):
      return 'public'

    return 'private'

In [27]:
data['school'] = data['school'].apply(classify_school)

In [28]:
data['school'].value_counts()

,count
school,
private,145
public,46
other,3


In [29]:
data['guardian_job'].value_counts()

,count
guardian_job,
تاجر,40
تاجر,7
موظف,6
عامل,6
طبيب,5
ربة بيت,4
محاسب,4
عامل,4
فني اسنان,3


In [ ]:
def classify_job(job):

    if pd.isna(job):
        return 'other'

    if (
        'موظف' in job or 'عامل' in job or 'مندوب' in job or
        'أستاذ' in job or 'جامعي' in job or 'حكومي' in job or
        'معلم' in job or 'مسؤول ' in job or 'supervisor' in job or
        'جامعة' in job or 'محاسب' in job or 'استاذ' in job or
        'مرشد' in job or 'محاضر' in job
    ):
        return 'employed'

    if (
        'بيت' in job or 'منزل' in job or 'لاشي' in job or 'سجون' in job or '.' in job or 'مواطن' in job
    ):
        return 'unemployed'

    return 'self_employed'

In [ ]:
data['guardian_job'] = data['guardian_job'].apply(classify_job)

In [ ]:
data['guardian_job'].value_counts()

In [ ]:
data['guardian_job_location'].value_counts()

In [ ]:
def classify_job_location(location):

# near: inside th city. far: outside the city.
    if pd.isna(location):
        return 'other'

    if (
        'دورا' in location or 'حلحول' in location or 'تفوح' in location or 'الظاهريه' in location or 'السعودية' in location or
         'سعير' in location or 'خرسا' in location or 'امريكا' in location or 'عسقلان' in location or 'رام الله' in location or
         'دوره' in location or 'بيت أمر' in location or 'ترقوميا' in location or 'بيت لحم' in location or 'بيت كاحل' in location or
         'بيت فجار' in location or 'الخط الاخضر' in location or 'عصيون' in location
    ):
        return 'far'

    return 'near'

In [ ]:
data['guardian_job_location'] = data['guardian_job_location'].apply(classify_job_location)

In [ ]:
data['guardian_job_location'].value_counts()

In [ ]:
data['previous_fighting_experience'].value_counts()

In [ ]:
def classify_previous_experience(answer):
# has experience: 1. Does not have: 0. Unknown: -1
    if pd.isna(answer):
        return -1

    if ('لم يمارس' in answer):
        return 0

    return 1

In [ ]:
data['previous_fighting_experience'] = data['previous_fighting_experience'].apply(classify_previous_experience)

In [ ]:
data['previous_fighting_experience'].value_counts()

In [ ]:
data['has_health_condition'].value_counts()

In [ ]:
def classify_helath_case(condition):
# has health conditions: 1. Does not have health conditions: 0. Unknown: -1
    if pd.isna(condition):
        return -1

    if ('جيدة' in condition):
        return 0

    return 1

In [ ]:
data['has_health_condition'] = data['has_health_condition'].apply(classify_helath_case)

In [ ]:
data['has_health_condition'].value_counts()

In [ ]:
data['had_surgery'].value_counts()

In [ ]:
def classify_surgery_history(surgery):
# had surgery: 1. Did not have surgery: 0. Unknown: -1
    if pd.isna(surgery):
        return -1

    if ('لا' in surgery):
        return 0

    return 1

In [ ]:
data['had_surgery'] = data['had_surgery'].apply(classify_surgery_history)

In [ ]:
data['had_surgery'].value_counts()

In [ ]:
data['needs_transportaion'].value_counts()

In [ ]:
def classify_transportaion_need(transport):
# needs transportaion: 1. Does not need transportaion: 0.

    if pd.isna(transport):
        return 0

    if ('لا' in transport):
        return 0

    return 1

In [ ]:
data['needs_transportaion'] = data['needs_transportaion'].apply(classify_transportaion_need)

In [ ]:
data['needs_transportaion'].value_counts()

In [ ]:
data['needs_transportaion'].value_counts()

In [ ]:
print('the shape of the data is:', data.shape)
data.head(25)

In [ ]:
#data['birthday'] = pd.to_datetime(data['birthday'], errors='coerce')

In [ ]:
#data['birthday'].value_counts()

### Feature Engineering

In [ ]:
# player_status: may be  stopped(0), continuous(1), or swindler(2)
"""data['player_status'] = data['player_status'].str.lower().map({
    'متوقف': 0,
    'مستمر': 1,
    'نصاب': 2
})"""